# **Text Recognition**

Setting up

Intalling tesseract for extracting text

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,742 kB/s)
debconf: unable to initi

Because file is stored in drive, we have to mount drive first.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!which tesseract #to know the dir for Pytesseract is stored

Installing keras and numpy

In [2]:
!pip install keras
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import libraries and display image file

In [ ]:
import pytesseract
import cv2
import matplotlib.pylab as plt

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract' #we know from the !which tesseract command

img = cv2.imread('/content/drive/MyDrive/Bangkit - Capstone/img_for_testing/test2.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

plt.title('Show Image')
plt.imshow(img)

In [ ]:
#to know the image size
hImg, wImg,_ = img.shape
img.shape

**Image Cleaning** 

In [ ]:
img = cv2.bilateralFilter(img, 5, 55, 60)
plt.imshow(img)


In [ ]:
#Text detection

detect = (pytesseract.image_to_data(img))
for x, detect in enumerate(detect.splitlines()):
  if x != 0:
    detect = detect.split()
    print(detect)
    if len(detect) == 12:
      x, y, w, h = int(detect[6]), int(detect[7]), int(detect[8]), int(detect[9])
      cv2.rectangle(img, (x,y), (w+x, h+y), (0,0,255), 2)
      #cv2.putText(img, detect[11], (x,y), cv2.FONT_HERSHEY_PLAIN, 1, (50,50,255), 3)

print('Text detection')
plt.imshow(img)

In [ ]:
#Recognize text

img_to_char = pytesseract.image_to_string(img)
print(img_to_char)

**Prediction**

In [ ]:
import csv
import random
import pickle
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from scipy.stats import linregress
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
NUM_WORDS = 1000
EMBEDDING_DIM = 100
MAXLEN = 16
PADDING = 'post'
OOV_TOKEN = "<OOV>"
TRAINING_SPLIT = .8
TRUNCATING = 'post'
MAX_EXAMPLES = 160000

In [ ]:
def fit_tokenizer(train_sentences, num_words, oov_token):
  tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV_TOKEN)
  tokenizer.fit_on_texts(train_sentences)
  return tokenizer

In [ ]:
def seq_and_pad(sentences, tokenizer, padding, maxlen):
  sequences = tokenizer.texts_to_sequences(sentences)
  padded_sequences = pad_sequences(sequences, maxlen=MAXLEN, padding=PADDING)
  
  return padded_sequences

In [ ]:
text = img_to_char
text = text.lower()
text1 = text.split()

In [ ]:
tokenizer = fit_tokenizer(text, NUM_WORDS, OOV_TOKEN)
word_index = tokenizer.word_index

print(f"Vocabulary contains {len(word_index)} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")

In [ ]:
new_model = tf.keras.models.load_model("/content/drive/MyDrive/Bangkit - Capstone/model.h5")
new_model.summary()

In [ ]:
for x in range(len(text1)) :
  word_token = seq_and_pad(text1[x], tokenizer, PADDING, MAXLEN)
  predict = new_model.predict(word_token)

In [ ]:
print(predict)